An example of table history

In [ ]:
%pip install -qU pixeltable

Let's insert a table and make some modifications to it

In [ ]:
import pixeltable as pxt
t = pxt.create_table(
    'test',
    {'c1': pxt.Required[pxt.Int], 'c2': pxt.String},
    comment='some random table comment',
    primary_key=['c1'],
    if_exists='replace_force'
)
t.insert([{'c1': 1, 'c2': 'a'}, {'c1': 2, 'c2': 'b'}])
t.add_computed_column(c3=t.c1 + 10)
t.add_computed_column(c4=t.c2.upper())
t.add_computed_column(c5=t.c1 + 10)
t.add_columns({'c6': pxt.String, 'c7': pxt.Int})
t.insert([{'c1': 3, 'c2': 'c'}, {'c1': 4, 'c2': 'd'}])
t.insert([{'c1': 5, 'c2': 'e'}, {'c1': 6, 'c2': 'e'}])
t.insert([{'c1': 7, 'c2': 'a'}, {'c1': 8, 'c2': 'b'}])
t.delete((t.c1 >= 4) & (t.c1 <= 5))
t.batch_update([{'c1': 2, 'c2': 'xxx'}])
t.rename_column('c2', 'c2_renamed')
t.drop_column('c4')

This is a lot of modifications and will create a fairly complex history.
Let's look at all of it:

In [ ]:
t.history()

If we're only interested in the last few modifications, we can request information about only those:

In [ ]:
t.history(max_versions=4)

If we are interested in details about the data changes, we can request that information as well:

In [ ]:
t.history(summarize_data_changes=True)

The information from a table history can be inserted into pandas or a pixeltable table for further filtering or other examination:

In [ ]:
th=pxt.create_table('t_history', source=t.history(summarize_data_changes=True).to_pandas(), if_exists='replace_force')

We can the summarize all of the data changes over the life of the table:

In [ ]:
import pixeltable.functions as pxtf
th.select(total_inserted=pxtf.sum(th.inserted), total_deleted=pxtf.sum(th.deleted), total_updated=pxtf.sum(th.updated)).collect()

If we create a view on the table, we can look at the history of the view:

In [ ]:
v = pxt.create_view('view1', t)
v.history()

And a snaphot on top of the view:

In [ ]:
s = pxt.create_snapshot('snapshot1', v)
s.history()

Let's do two inserts into the original table and look at the new history:

In [ ]:
t.insert([{'c1': 9, 'c2_renamed': 'a'}, {'c1': 10, 'c2_renamed': 'b'}])
t.insert([{'c1': 9, 'c2_renamed': 'a'}, {'c1': 10, 'c2_renamed': 'b'}])
t.history()

In [ ]:
v.history()

In [ ]:
s.history()